In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from GenDARA.dsp import calculate_rt, calculate_drr, calculate_edf, load_rirs

%load_ext autoreload
%autoreload 2

# Evaluate accuracy of simulated RIRs in Room 0
Room 0 is the control/calibration room measured at Treble's office. 

The provided data from Room 0 includes the: 
- 3D model
- 20 measured RIRs (2 sources, 20 receivers) [mono]
- 20 corresponding simulated RIRs [mono and 8th order HOA]
- 404 simulated RIRs from a grid receivers (2 sources x 202 receivers) [mono and 8th order HOA]

Below, we calculate the T20 Percentage Error, EDF (dB) Mean Squared Error, and Direct to Reverberant Ratio (dB) Mean Squared Error. Results reported in table III in the paper.

### Calculate RT, EDF, and DRR error
#### T20 Percentage Error

In [4]:
config = {'sr': 32000,
        'filter_frequencies': [125, 250, 500, 1000, 2000, 4000],
        'db_high_cutoff': 0,
        'db_low_cutoff': 20
}

dir = './enrollment_data/Room_0_data/measured_rirs'
######### load ground truth metadata to dataframe
df1 = pd.read_csv(os.path.join(dir, 'meta.csv'))
# load ground truth (measured) RIRs
y_meas = load_rirs(dir, os.path.join(dir, "meta.csv"))

# calculate reverberation time for the fullband and per octave band
df1['rt'] = calculate_rt(y_meas, per_octaveband=False, **config)
df1[['rt_{}'.format(f) for f in config['filter_frequencies']]] = calculate_rt(y_meas, per_octaveband=True, **config)

######### Load simulated metadata to dataframe
dir = './enrollment_data/Room_0_data/simulated_rirs/measurement_positions/mono'
df2 = pd.read_csv(os.path.join(dir, 'meta.csv'))
# load ground truth (measured) RIRs
y_sim = load_rirs(dir, os.path.join(dir, "meta.csv"))

df2['rt'] = calculate_rt(y_sim, per_octaveband=False, **config)
df2[['rt_{}'.format(f) for f in config['filter_frequencies']]] = calculate_rt(y_sim, per_octaveband=True, **config)


######### merge df1 and df2 based on index.
df_error = pd.merge(df1, df2[['rt'] + ['rt_{}'.format(f) for f in config['filter_frequencies']]], left_index=True, right_index=True, suffixes=('_df1', '_df2'))
print(df_error.columns)

# calculate the percentage relative error
df_error['T60_PE'] = np.abs((df_error['rt_df1'] - df_error['rt_df2']))/ df_error['rt_df1']
for f in config['filter_frequencies']:
    df_error['T60_PE_{}'.format(f)] = np.abs((df_error['rt_{}_df1'.format(f)] - df_error['rt_{}_df2'.format(f)]))/ df_error['rt_{}_df1'.format(f)]

df_error = df_error.drop(columns=df_error.filter(regex='rt').columns)
df_error.mean(numeric_only=True)

Loaded dataset shape: (20, 12800)
Loaded dataset shape: (20, 12800)
Index(['roomID', 'receiverType', 'rirType', 'label_r', 'x_r', 'y_r', 'z_r',
       'label_s', 'x_s', 'y_s', 'z_s', 'dist_gt', 'filename', 'id', 'rt_df1',
       'rt_125_df1', 'rt_250_df1', 'rt_500_df1', 'rt_1000_df1', 'rt_2000_df1',
       'rt_4000_df1', 'rt_df2', 'rt_125_df2', 'rt_250_df2', 'rt_500_df2',
       'rt_1000_df2', 'rt_2000_df2', 'rt_4000_df2'],
      dtype='object')


x_r            3.438000
y_r            2.177000
z_r            1.426500
x_s            3.745000
y_s            2.935000
z_s            1.630000
dist_gt        2.852680
T60_PE         0.134635
T60_PE_125     0.100867
T60_PE_250     0.149968
T60_PE_500     0.109062
T60_PE_1000    0.120630
T60_PE_2000    0.069308
T60_PE_4000    0.177995
dtype: float64

#### Energy Decay Function (EDF) Mean Squared Error

In [5]:
edf_meas = calculate_edf(y_meas, per_octaveband=False, plot=False, **config)
edf_sim = calculate_edf(y_sim, per_octaveband=False, plot=False, **config)

def plot_edfs():
    # plot corresponding edfs from edf_meas and edf_sim for comparison 
    plt.figure()
    for i in range(len(edf_meas)):
        plt.plot(edf_meas[i,:], label='measured')
        plt.plot(edf_sim[i,:], label='simulated')
        plt.legend()
        plt.title(f'EDF comparison: {i}') 
        plt.show()
       
def plot_waveforms(): 
    # plot corresponding edfs from edf_meas and edf_sim for comparison 
    plt.figure()
    idx = -3000
    for i in range(len(edf_meas)):
        plt.plot(y_meas[i,idx:], label='measured')
        plt.plot(y_sim[i,idx:], label='simulated')
        plt.legend()
        plt.title(f'Pressure comparison: {i}') 
        plt.show()

# plot_edfs()
# plot_waveforms()
########## Full band EDF MSE
edf_mse = np.empty(edf_meas.shape[0])
for i in range(len(edf_meas)):
    gt = edf_meas[i,:]
    sim = edf_sim[i,:]
    error = np.mean((gt-sim)**2)
    edf_mse[i] = error.item()
# print(edf_mse)

# Drop last 5% of the EDF values, as they are not reliable
edf_mse = np.empty(edf_meas.shape[0])
for i in range(len(edf_meas)):
    gt = edf_meas[i,:-int(0.05*len(edf_meas[i,:]))]
    sim = edf_sim[i,:-int(0.05*len(edf_meas[i,:]))]
    error = np.mean((gt-sim)**2)
    edf_mse[i] = error.item()
# print(edf_mse)


######### Octave band EDF MSE
edf_meas_band = calculate_edf(y_meas, per_octaveband=True, plot=False, **config)
edf_sim_band = calculate_edf(y_sim, per_octaveband=True, plot=False, **config)

edf_mse_per_band = np.empty((len(edf_meas_band), len(config['filter_frequencies'])))
for j, f in enumerate(config['filter_frequencies']):
    for i in range(len(edf_meas_band)):
        gt = edf_meas_band[i,j,:]
        sim = edf_sim_band[i,j,:]
        error = np.mean((gt-sim)**2)
        edf_mse_per_band[i,j] = error.item()
# print(edf_mse_per_band)
# print("\n")

edf_mse_per_band = np.empty((len(edf_meas_band), len(config['filter_frequencies'])))
for j, f in enumerate(config['filter_frequencies']):
    for i in range(len(edf_meas_band)):
        gt = edf_meas_band[i,j,:-int(0.05*len(edf_meas[i,:]))]
        sim = edf_sim_band[i,j,:-int(0.05*len(edf_meas[i,:]))]
        error = np.mean((gt-sim)**2)
        edf_mse_per_band[i,j] = error.item()
        
# print(edf_mse_per_band)
df_error['EDF_MSE'] = edf_mse
df_error[['EDF_MSE_{}'.format(f) for f in config['filter_frequencies']]] = edf_mse_per_band

df_error.mean(numeric_only=True)

x_r              3.438000
y_r              2.177000
z_r              1.426500
x_s              3.745000
y_s              2.935000
z_s              1.630000
dist_gt          2.852680
T60_PE           0.134635
T60_PE_125       0.100867
T60_PE_250       0.149968
T60_PE_500       0.109062
T60_PE_1000      0.120630
T60_PE_2000      0.069308
T60_PE_4000      0.177995
EDF_MSE         22.014560
EDF_MSE_125      7.032809
EDF_MSE_250      7.607006
EDF_MSE_500      5.012496
EDF_MSE_1000     4.410310
EDF_MSE_2000     7.702280
EDF_MSE_4000    41.084141
dtype: float64

#### DRR MSE

In [ ]:
drr_meas = calculate_drr(y_meas, per_octaveband=False, plot=False, **config)
drr_sim = calculate_drr(y_sim, per_octaveband=False, plot=False, **config)

drr_se = (drr_meas - drr_sim)**2
df_error['DRR_SE'] = drr_se

drr_mse = np.mean(drr_se)
print(f'DRR_MSE: {drr_mse}')

DRR_MSE: 14.466146469116211
